In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '4.2'
var_number = 1


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        C = fromCopyString(variant[2])
        P = fromCopyString(variant[3])
        F = fromCopyString(variant[4])
    return A, B, C, P, F

In [109]:
A, B, C, P, F = получить_матрицы_варианта(var_number)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
C = sympy.Matrix(C)
P = sympy.Matrix(P)
F = sympy.Matrix(F)
P

Matrix([[-8, -5, 0, 0]])

In [146]:
def solve(A, B, C, P, F):
    # Матрица наблюдаемости
    observability_matrix = sympy.Matrix([(C.T).T, (A.T*C.T).T]).T

    # Определитель матрицы наблюдаемости
    determinant = observability_matrix.det()
    
    # x = [x1, x2, x3, x4]
    x = sympy.symbols('x1:5')
    
    res = sympy.Matrix([0, 0, 0, 0])
    no_res = sympy.Matrix([0, 0, 0, 0])
    for i in range(4):
        if P[i] == 0:
            x = sympy.Symbol(f'x{i+1}')
            res[i] = x
        else:
            x = sympy.Symbol(f'x{i+1}')
            no_res[i] = x
        
    # Матрица T
    p1, p2, p3, p4 = sympy.symbols('p1 p2 p3 p4')
    H = sympy.Matrix([sympy.symbols('h1'), sympy.symbols('h2'), sympy.symbols('h3'), sympy.symbols('h4')])
    T = sympy.Matrix([sympy.symbols('t1'), sympy.symbols('t2'), sympy.symbols('t3'), sympy.symbols('t4')])
    for i in range(4):
        if not (C[0, i] != 0 or C[1, i] != 0):
            T[i] = P[i]
            H[i] = 0
    
    T = T.T
    H = H.T


    eq = T*A - F*T - H
    # solve eq for t1, t2, t3, t4, h1, h2, h3, h4
    sol = sympy.solve(eq, [
        sympy.symbols('t1'), sympy.symbols('t2'), sympy.symbols('t3'), sympy.symbols('t4'), 
        sympy.symbols('h1'), sympy.symbols('h2'), sympy.symbols('h3'), sympy.symbols('h4')
    ])

    for key in sol.keys():
        if key == sympy.symbols('t1'):
            t1 = sol[key]
        elif key == sympy.symbols('t2'):
            t2 = sol[key]
        elif key == sympy.symbols('t3'):
            t3 = sol[key]
        elif key == sympy.symbols('t4'):
            t4 = sol[key]
        elif key == sympy.symbols('h1'):
            h1 = sol[key]
        elif key == sympy.symbols('h2'):
            h2 = sol[key]
        elif key == sympy.symbols('h3'):
            h3 = sol[key]
        elif key == sympy.symbols('h4'):
            h4 = sol[key]
    
    for i in range(4):
        if T[i] != P[i]:
            if i == 0:
                T[i] = t1
            elif i == 1:
                T[i] = t2
            elif i == 2:
                T[i] = t3
            elif i == 3:
                T[i] = t4
        if H[i] != 0:  
            if i == 0:
                H[i] = h1
            elif i == 1:
                H[i] = h2
            elif i == 2:
                H[i] = h3
            elif i == 3:
                H[i] = h4
    
    T = T.n()
    H = H.T.n()

    TB = T*B
    
    result_string_for_z_der = (f"z' = {F[0]} z + {H[0]} x_1 + {H[1]} x_2 + {H[2]} x_3 + {H[3]} x_4 + {TB[0]} u_1 + {TB[1]} u_2")
    result_string_for_w = 'w = '
    flag = True
    for i in range(4):
        if sympy.symbols(f'x{i+1}') in no_res:
            result_string_for_w += (f"{T[i]} x_{i+1}")
            if flag:
                result_string_for_w += ' + '
                flag = False
                
    result_string_for_w += ' = z'
    for i in range(4):
        if sympy.symbols(f'x{i+1}') in res:
            result_string_for_w += (f" - {T[i]} x_{i+1}")

    # print(f"z = {T[0]} x_1 + {T[1]} x_2 + {T[2]} x_3 + {T[3]} x_4")


    return observability_matrix, determinant, T, H, result_string_for_z_der, result_string_for_w

In [147]:
solve(A, B, C, P, F)

(Matrix([
 [0, 0, -5,  -6],
 [0, 0,  4,  -8],
 [0, 1, -6,   8],
 [1, 0, -8, -10]]),
 -64,
 Matrix([[-2.0, -3.0, -3.390625, -2.53125]]),
 Matrix([
 [         0],
 [         0],
 [-27.671875],
 [   45.4375]]),
 "z' = -7 z + 0 x_1 + 0 x_2 + -27.6718750000000 x_3 + 45.4375000000000 x_4 + -12.9531250000000 u_1 + -11.6562500000000 u_2",
 'w = -2.00000000000000 x_1 + -3.00000000000000 x_2 = z - -3.39062500000000 x_3 - -2.53125000000000 x_4')

In [148]:
for var_number in range(1, 31):
    A, B, C, P, F = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    C = sympy.Matrix(C)
    P = sympy.Matrix(P)
    F = sympy.Matrix(F)
    try:
        observability_matrix, determinant, T, H, result_string_for_z_der, result_string_for_w = solve(A, B, C, P, F)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(observability_matrix) + '\n')
            file.write(str(determinant) + '\n')
            file.write(toCopyString(T) + '\n')
            file.write(toCopyString(H) + '\n')
            file.write(result_string_for_z_der + '\n')
            file.write(result_string_for_w + '\n')
    # catch error
    except Exception as e:
        print(f'Error in {var_number}: {e}')   
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
